In [ ]:
#Face Recognition

In [8]:
import cv2
import os
import dlib
import numpy as np
video_file='Output/v5.mov'
output_dir="Output/Result"
os.makedirs(output_dir,exist_ok=True)
cap=cv2.VideoCapture(video_file)
if not cap.isOpened():
    print("Error:Could not open video file")
    exit()

face_rec_model = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1/dlib_face_recognition_resnet_model_v1.dat")
face_detector = dlib.get_frontal_face_detector()
input_image = cv2.imread("Output/inputperson1.png")
input_image_rgb = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
input_faces = face_detector(input_image_rgb)

if len(input_faces) == 0:
    print("No faces found in the input image.")
else:
    landmarks = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")(input_image_rgb, input_faces[0])
    input_face_descriptor = face_rec_model.compute_face_descriptor(input_image_rgb, landmarks)

frame_rate=int(cap.get(cv2.CAP_PROP_FPS))
frame_count=0
frame_number=1

while True:
    ret,frame=cap.read()
    if not ret:
        break
    
    frame_count+=1
    if frame_count%frame_rate==0:
        frame_name = f"{output_dir}/frame_{frame_number:04d}.jpg"
        frame_number += 1
        frame_faces=face_detector(frame)
        for face in frame_faces:
            landmarks=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")(frame,face)
            frame_descriptor=face_rec_model.compute_face_descriptor(frame,landmarks)
            distance = np.linalg.norm(np.array(input_face_descriptor) - np.array(frame_descriptor))
            if distance < 0.4:
#                 output_path = os.path.join(output_dir, frame_filename)
                x, y, w, h = face.left(), face.top(), face.width(), face.height()
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.imwrite(frame_name, frame)
                print(f"Match found in {frame_name},{distance}")
            else:
                 print("Match not found")
                 
            
        

In [ ]:
#Clothing Color Analysis

In [7]:
import cv2
import numpy as np

# Load the input image
input_image = cv2.imread('Output/inputperson1.png')
output_dir="Output/Clothing"
# Create a histogram for the input image
input_hist = cv2.calcHist([input_image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
input_hist = cv2.normalize(input_hist, input_hist).flatten()

# Open the video file
video_capture = cv2.VideoCapture('Output/v5.mov')
frame_rate=int(video_capture.get(cv2.CAP_PROP_FPS))
frame_count=0
frame_number=1
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

# Load COCO class names
with open('coco.names', 'r') as f:
    classes = f.read().strip().split('\n')

# Define the output layer names
output_layers = net.getUnconnectedOutLayersNames()


while True:
    ret, frame = video_capture.read()

    if not ret:
        break

    frame_count+=1
    if frame_count%frame_rate==0:
        frame_name = f"{output_dir}/frame_{frame_number:04d}.jpg"
        frame_number += 1
        
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id == 0:  # Class ID 0 corresponds to 'person'
                    center_x = int(detection[0] * frame.shape[1])
                    center_y = int(detection[1] * frame.shape[0])
                    w = int(detection[2] * frame.shape[1])
                    h = int(detection[3] * frame.shape[0])

                    # Calculate the coordinates of the bounding box
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Non-maximum suppression to remove duplicate detections
        indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        for i in range(len(boxes)):
            if i in indices:
                x, y, w, h = boxes[i]

                # Extract the region of interest (ROI) around the detected person
                person_roi = frame[y:y + h, x:x + w]

                # Calculate a color histogram for the detected person
                person_hist = cv2.calcHist([person_roi], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
                person_hist = cv2.normalize(person_hist, person_hist).flatten()
                
                bhattacharyya_distance = cv2.compareHist(input_hist, person_hist, cv2.HISTCMP_BHATTACHARYYA)

                threshold=0.4
                # If the distance is below the threshold, save the frame
                if bhattacharyya_distance < threshold:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 8)
                    cv2.imwrite(frame_name, frame)
                    print(f"Match found in {frame_name}")
#                 else:
#                      print("Match not found")



# Release the video capture and writer
video_capture.release()
cv2.destroyAllWindows()


Match found in Output/Clothing/frame_0001.jpg
Match found in Output/Clothing/frame_0003.jpg
Match found in Output/Clothing/frame_0004.jpg
Match found in Output/Clothing/frame_0005.jpg
Match found in Output/Clothing/frame_0135.jpg
Match found in Output/Clothing/frame_0163.jpg
Match found in Output/Clothing/frame_0164.jpg
Match found in Output/Clothing/frame_0174.jpg
Match found in Output/Clothing/frame_0280.jpg
Match found in Output/Clothing/frame_0310.jpg
Match found in Output/Clothing/frame_0332.jpg
Match found in Output/Clothing/frame_0366.jpg
Match found in Output/Clothing/frame_0367.jpg
Match found in Output/Clothing/frame_0573.jpg
Match found in Output/Clothing/frame_0628.jpg
Match found in Output/Clothing/frame_0629.jpg


In [ ]:
#Overall

In [ ]:
import cv2
import os
import dlib
import numpy as np

# Set the paths for the input files and the output directory
video_file = 'Output/v5.mov'
input_face_image = 'Output/inputperson1.png'
output_dir = "Output/Result"
os.makedirs(output_dir, exist_ok=True)

# Initialize face recognition models
face_rec_model = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1/dlib_face_recognition_resnet_model_v1.dat")
face_detector = dlib.get_frontal_face_detector()
input_image = cv2.imread(input_face_image)
input_image_rgb = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
input_faces = face_detector(input_image_rgb)

if len(input_faces) == 0:
    print("No faces found in the input image.")
else:
    landmarks = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")(input_image_rgb, input_faces[0])
    input_face_descriptor = face_rec_model.compute_face_descriptor(input_image_rgb, landmarks)

# Clothing matching using YOLO object detection
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')
with open('coco.names', 'r') as f:
    classes = f.read().strip().split('\n')
output_layers = net.getUnconnectedOutLayersNames()

# Open video file for processing
video_capture = cv2.VideoCapture(video_file)
frame_rate = int(video_capture.get(cv2.CAP_PROP_FPS))
frame_count = 0
frame_number = 1

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % frame_rate == 0:
        frame_name = f"{output_dir}/frame_{frame_number:04d}.jpg"
        frame_number += 1

        # Initialize variables to keep track of matches for the entire frame
        match_found = False

        # Face recognition part
        frame_faces = face_detector(frame)
        for face in frame_faces:
            landmarks = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")(frame, face)
            frame_descriptor = face_rec_model.compute_face_descriptor(frame, landmarks)
            distance = np.linalg.norm(np.array(input_face_descriptor) - np.array(frame_descriptor))

        # YOLO object detection for clothing matching
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id == 0:  # Class ID 0 corresponds to 'person'
                    center_x = int(detection[0] * frame.shape[1])
                    center_y = int(detection[1] * frame.shape[0])
                    w = int(detection[2] * frame.shape[1])
                    h = int(detection[3] * frame.shape[0])

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    person_roi = frame[y:y + h, x:x + w]

                    person_hist = cv2.calcHist([person_roi], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
                    person_hist = cv2.normalize(person_hist, person_hist).flatten()

                    bhattacharyya_distance = cv2.compareHist(input_hist, person_hist, cv2.HISTCMP_BHATTACHARYYA)
                    threshold = 0.4

                    if bhattacharyya_distance < threshold and distance < threshold:
                        match_found = True

        # Check if any match is found for the entire frame
        if match_found:
            # Draw bounding box and save the frame
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 8)
            cv2.imwrite(frame_name, frame)
            print(f"Match found in {frame_name}, Face Distance: {distance}, Clothing Distance: {bhattacharyya_distance}")
        else:
            print("Match not found")

video_capture.release()
cv2.destroyAllWindows()
